In [2]:
import os
os.chdir('/content/cvip')

In [3]:
!pip install -q torch torchvision pycocotools
!git clone https://github.com/facebookresearch/detr.git
%cd detr
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.9 MB/s eta 0:00:00
Cloning into 'detr'...
remote: Enumerating objects: 265, done.
remote: Total 265 (delta 0), reused 0 (delta 0), pack-reused 265 (from 1)
Receiving objects: 100% (265/265), 21.19 MiB | 15.83 MiB/s, done.

In [22]:
%%writefile datasets/coco.py
from pathlib import Path
from PIL import Image
import torch
import torchvision.transforms as T
from torchvision.datasets import CocoDetection as TVCocoDetection
from torchvision.transforms import ToTensor

def make_coco_transforms(image_set):
    normalize = T.Compose([
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406],
                    [0.229, 0.224, 0.225])
    ])
    if image_set == 'train':
        return T.Compose([
            T.RandomHorizontalFlip(),
            normalize,
        ])
    return T.Compose([normalize])

class CocoDetection(TVCocoDetection):
    def __init__(self, img_folder, ann_file, transforms=None):
        super().__init__(img_folder, ann_file)
        self._transforms = transforms

    def __getitem__(self, idx):
        img, target = super().__getitem__(idx)

        w, h = img.size
        boxes = []
        labels = []
        image_id = self.ids[idx]
        for obj in target:
            bbox = obj["bbox"]
            x, y, bw, bh = bbox
            boxes.append([x, y, x + bw, y + bh])
            labels.append(obj["category_id"])
        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
            "image_id": torch.tensor(image_id),
            "orig_size": torch.tensor([h, w]),
            "size": torch.tensor([h, w])
        }

        img = ToTensor()(img)
        return img, target


def build(image_set, args):
    assert image_set in ['train', 'val', 'test']
    root = Path(args.dataset_path)
    PATHS = {
        "train": (root / "train", root / "train" / "_annotations.coco.json"),
        "val": (root / "valid", root / "valid" / "_annotations.coco.json"),
        "test": (root / "test", root / "test" / "_annotations.coco.json"),
    }
    img_folder, ann_file = PATHS[image_set]
    dataset = CocoDetection(img_folder, ann_file, transforms=make_coco_transforms(image_set))
    return dataset


Overwriting datasets/coco.py


In [5]:
%%writefile models/build.py
from models import build_model

def build_detr(args):
    args.num_classes -= 1
    model, criterion, postprocessors = build_model(args)
    return model, criterion, postprocessors


Writing models/build.py


In [46]:
%%writefile train_detr_players.py

import sys
sys.path.append("/content/cvip/detr")

import torch
from torch.utils.data import DataLoader
from datasets import coco
from models.builder_detr import build_detr
from engine import train_one_epoch, evaluate
import util.misc as utils
import argparse
import os

def get_args():
    parser = argparse.ArgumentParser('Train DETR', add_help=False)
    parser.add_argument('--dataset_path', default='/content/cvip/coco', type=str) #Give the path to the dataset folder
    parser.add_argument('--output_dir', default='./output', type=str)
    parser.add_argument('--num_classes', default=2, type=int)
    parser.add_argument('--epochs', default=25, type=int)
    parser.add_argument('--lr', default=1e-4, type=float)
    parser.add_argument('--lr_backbone', default=1e-5, type=float)
    parser.add_argument('--batch_size', default=2, type=int)
    parser.add_argument('--weight_decay', default=1e-4, type=float)
    parser.add_argument('--backbone', default='resnet50', type=str)
    parser.add_argument('--dilation', action='store_true')
    parser.add_argument('--position_embedding', default='sine', type=str)
    parser.add_argument('--masks', action='store_true')
    parser.add_argument('--device', default='cuda')
    parser.add_argument('--dataset_file', default='coco', type=str)
    parser.add_argument('--hidden_dim', default=256, type=int)
    parser.add_argument('--dropout', default=0.1, type=float)
    parser.add_argument('--nheads', default=8, type=int)
    parser.add_argument('--dim_feedforward', default=2048, type=int)
    parser.add_argument('--enc_layers', default=6, type=int)
    parser.add_argument('--dec_layers', default=6, type=int)
    parser.add_argument('--pre_norm', action='store_true')
    parser.add_argument('--num_queries', default=100, type=int)
    parser.add_argument('--aux_loss', action='store_true')
    parser.add_argument('--set_cost_class', default=1, type=float)
    parser.add_argument('--set_cost_bbox', default=5, type=float)
    parser.add_argument('--set_cost_giou', default=2, type=float)
    parser.add_argument('--bbox_loss_coef', default=5, type=float)
    parser.add_argument('--giou_loss_coef', default=2, type=float)
    parser.add_argument('--eos_coef', default=0.1, type=float)


    return parser.parse_args([])

def main():
    args = get_args()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset_train = coco.build('train', args)
    dataset_val = coco.build('val', args)

    sampler_train = torch.utils.data.RandomSampler(dataset_train)
    sampler_val = torch.utils.data.SequentialSampler(dataset_val)

    batch_sampler_train = torch.utils.data.BatchSampler(sampler_train, args.batch_size, drop_last=True)

    data_loader_train = DataLoader(dataset_train, batch_sampler=batch_sampler_train, collate_fn=utils.collate_fn)
    data_loader_val = DataLoader(dataset_val, batch_size=1, sampler=sampler_val, collate_fn=utils.collate_fn)

    model, criterion, postprocessors = build_detr(args)
    model.to(device)

    param_dicts = [
        {"params": [p for n, p in model.named_parameters() if "backbone" not in n and p.requires_grad]},
        {"params": [p for n, p in model.named_parameters() if "backbone" in n and p.requires_grad], "lr": args.lr * 0.1},
    ]
    optimizer = torch.optim.AdamW(param_dicts, lr=args.lr, weight_decay=args.weight_decay)

    os.makedirs(args.output_dir, exist_ok=True)
    for epoch in range(args.epochs):
        train_stats = train_one_epoch(model, criterion, data_loader_train, optimizer, device, epoch)

        base_ds = dataset_val.coco
        _, coco_evaluator = evaluate(model, criterion, postprocessors, data_loader_val, dataset_val,device, args.output_dir)
        if hasattr(coco_evaluator.coco_eval['bbox'], 'stats'):
          map_score = coco_evaluator.coco_eval['bbox'].stats[0]
          print(f"Epoch {epoch+1} mAP: {map_score:.4f}")
        else:
          print(f"Epoch {epoch+1}: No valid mAP score found.")
        #print(f"Epoch {epoch+1} mAP: {map_score:.4f}")
        #print(f"Epoch {epoch+1} mAP: {map_score:.4f}")

        checkpoint_path = f"{args.output_dir}/detr_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), checkpoint_path)

if __name__ == "__main__":
    main()


Overwriting train_detr_players.py


In [18]:
!mv /content/cvip/detr/models/detr_builder.py /content/cvip/detr/models/builder_detr.py

mv: cannot stat '/content/cvip/detr/models/detr_builder.py': No such file or directory


In [8]:
!mv /content/cvip/detr/models/build.py /content/cvip/detr/models/builder_detr.py


In [21]:
!touch /content/cvip/detr/models/__init__.py

In [ ]:
!sed -i 's/np\.float/float/g' /usr/local/lib/python3.11/dist-packages/pycocotools/cocoeval.py

In [24]:
%cd /content/cvip/detr
!python train_detr_players.py --dataset_path "/content/cvip/coco" --output_dir "./output" --num_classes 2